In [1]:
!jupyter nbextension enable --py widgetsnbextension
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np

import skimage

from sklearn.datasets import fetch_lfw_people
from sklearn.externals import joblib
from skimage.feature import hog
from sklearn.feature_extraction.image import extract_patches_2d
from skimage.io import imread
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

from tqdm import tqdm, tqdm_notebook, tnrange

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
people = fetch_lfw_people(color=True)

In [6]:
people.keys()
people.images.shape

(13233, 62, 47, 3)

In [7]:
filepath = "C:\\Users\\danai\\Projects\\python projects\\PycharmProjects\\IT_1_coursework\\neg_ex\\космос.png"
file_ = imread(filepath)
patches = extract_patches_2d(file_, people.images.shape[1:3], max_patches = people.images.shape[0])

In [8]:
print(file_.shape)
print(patches.shape)

(1956, 3840, 3)

(13233, 62, 47, 3)

In [11]:
people_fv = list()
for img in tqdm_notebook(people['images'], desc='processing positive examples'):
    #, orientations=8, pixels_per_cell=(6, 6), cells_per_block=(1, 1),feature_vector=True, visualise=True
    fv, hog_img = hog(img[:, :, 2], visualise=True)
    people_fv.append(fv)
    # print(fv.shape)
    # print(fv)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
    ax1.axis('off')
    ax1.imshow(img.astype(np.int))
    ax1.set_title('Input image')
    ax2.axis('off')
    ax2.imshow(hog_img, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()
    """
people_fv=np.asarray(people_fv)

C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


KeyboardInterrupt: 

In [ ]:
print(people_fv[0].shape)
print(len(people_fv))

(560,)
13233


In [ ]:
patch_fv = list()
for patch in tqdm_notebook(patches, desc='processing negative examples'):
    #, orientations=8, pixels_per_cell=(6, 6), cells_per_block=(1, 1),feature_vector=True, visualise=True
    fv, hog_patch = hog(patch[:, :, 2], visualise=True)
    patch_fv.append(fv)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
    ax1.axis('off')
    ax1.imshow(patch)
    ax1.set_title('Input image')
    ax2.axis('off')
    ax2.imshow(hog_patch,  cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()
    """
patch_fv = np.asarray(patch_fv)

C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [ ]:
label=np.hstack((np.ones(len(people_fv)), np.zeros(len(patch_fv))))
all_ex=np.vstack((people_fv, patch_fv))
print(all_ex.shape)
print(label.shape)

X_train, X_test, y_train, y_test = train_test_split(all_ex, label, shuffle=True, random_state=42)

print(y_test.mean())
print(y_test.std())

(26466, 560)
(26466,)
0.5052138431313284
0.49997281510078323


In [10]:
cls = LinearSVC(dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True,
                intercept_scaling=1, class_weight=None, verbose=0, random_state=42, max_iter=1000)
cls.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [11]:
x_pred=cls.predict(X_test)
x_pred

array([0., 0., 1., ..., 1., 0., 0.])

In [12]:
print(f"score:", cls.score(X_test, y_test))
avrg = ['micro', 'macro', 'weighted', None]
for a in avrg:
    print(f"{a}: {f1_score(y_test, x_pred, average=a)}")

score: 0.9916880761674475
micro: 0.9916880761674475
macro: 0.9916876744543999
weighted: 0.9916882770239712
None: [0.99162989 0.99174546]


In [13]:
print(people.images.shape[1:3])
joblib.dump(cls, 'cls.pkl')

(62, 47)


['cls.pkl']